## Interface 

In this notebook we automatise the computation of matrix elements for later use by a Krylov iterative solver. We save all symbolic results in a file containing reusable functions:

    interfaceME.py
    
This file only has to be generated once (per Green's function. 

In [1]:
from sympy import *
import numpy as np
import re

### Definitions and auxiliary functions

In [2]:
x1, y1, z1, x2, y2, z2 = symbols("x1, y1, z1, x2, y2, z2", real=True)
xij, yij, zij, dzij, zi = symbols("xij, yij, zij, dzij, zi", real=True)

b, h, eta = symbols("b, h, eta", real=True, positive=True)
theta = symbols("theta", real=True) ##eventually rename theta->psi


mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma = symbols(
    "mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma", integer=True)

R1 = Array([x1,y1,z1]) #why not use matrix here?
R2 = Array([x2,y2,z2])

KroneckerDelta = eye(3)

M  = Array([[1,0,0],[0,1,0],[0,0,-1]]) #diag(1,1,-1)

Mf = Array([[(1-theta)/(1+theta),0,0], [0, (1-theta)/(1+theta),0], [0,0,-1]])
#diag( (1-theta)/(1+theta), (1-theta)/(1+theta), -1) 

R2M = tensorcontraction(tensorproduct(M,R2), (1,2))

rM = R1 - R2M

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)#sqrt(summation(r[alpha]**2,(alpha,0,2)))

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Laplacian(G, r):
    return diff(G, r[0], r[0]) + diff(G, r[1], r[1]) + diff(G, r[2], r[2])


## for some reason I cannot sum over 'symbols' here whenever rM is a Matrix, not an Array. Who knows why, but this seems to work. 

def Curl(mu, beta): ##Curl is always with respect to R1
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(mu,i,j)*diff(G(j,beta),R1[i]) ## Need dependence of G on rM and R2? 
    return result

def tilde_Curl(alpha, sigma): ##tilde_Curl is always with respect to R2
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(sigma,i,j)*diff(G(alpha, j),R2[i])
    return result

def CurlCurl(mu, sigma):
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(mu, i, j)*eps(sigma, k, l)*diff(G(j, l), R1[i], R2[k])
    return result

In [4]:
def Fl(G, l):
    return G + b**2/(4*l+6)*Laplacian(G, R1)

def tilde_Fl(G,l):
    return G + b**2/(4*l+6)*Laplacian(G, R2)

def FlFl(G, l, lprime):
    return G + b**2/(4*l+6)*Laplacian(G, R1) + b**2/(4*lprime+6)*Laplacian(G, R2) + b**4/((4*l+6)*(4*lprime+6))*Laplacian(Laplacian(G, R1), R2)

In [5]:
## Green's function
##first G* part, which is the same for ii and ij, later G0 part for ii

def GO(r, alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(r) + r[alpha]*r[beta]/norm(r)**3) ##can reuse for particle-particle interactions

## only for Gstar, particle - image interactions
def Stokeslet(alpha, beta):
    result = 0
    for i in range(3):
        result += Mf[beta, i]*GO(rM, alpha, i)
    return result

def Dipole(alpha, beta):
    result = 0
    for i in range(3):
        result += -2*R2[2]*theta/(1+theta)*diff(GO(rM, alpha, 2) * M[beta, i], R1[i])
    return result

def Quadrupole(alpha, beta):
    result = 0
    for i in range(3):
        result += R2[2]**2*theta/(1+theta)*Laplacian(GO(rM, alpha, i) * M[beta, i], R1)
    return result

def G(alpha, beta): #introduce flag to switch between G0 and Gstar, for whichever we have to compute the matrix elements, also include flag for wall, since theta=infty is hard to handle
    return Stokeslet(alpha, beta) + Dipole(alpha, beta) + Quadrupole(alpha, beta)

In [6]:
## is there not a smarter way to do this? No dependence on rM and R2.. in python want xij, yij etc not x1- x2
## Need: self-interaction and ineraction with image. Interaction with other particle and with its image
## constant, R1=R2 (also constant), rij, and also rij. How to compute all of them with same code? 

## keep dependence on rM and R2: 
        ## for self-interaction choose rM = 0, 0, 2h and R2= 0,0,h
        ## for interaction choose: x1-x2=xij with particle, z1+z2 = ?

## keep dependence for now, might give greater flexibility later - no, delete to make shorter and write extra functions for particle-particle interactions
#for interactions, replace rM by xij, yij, dz = z1 + z2 or so, but there is also zij for particle-particle interactions, need another Greens function GO(rij, alpha, beta)

## could use flag in G, whether to return Go or Gstar

In [7]:
def G1s1s(alpha, beta):
    return FlFl( G(alpha, beta), 0, 0)

def G2a1s(mu, beta):
    return b*tilde_Fl(Curl(mu, beta), 0)

def G1s2a(alpha,sigma):
    return 0.5*b*Fl(tilde_Curl(alpha, sigma), 0)

def G2a2a(mu, sigma):
    return 0.5*b**2*CurlCurl(mu, sigma)

def G1s2s(alpha, sigma, eta1):
    return 0.5*b*FlFl((diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma])), 0, 1)

def G2a2s(mu, sigma, eta1):
    return 0.5*b**2*tilde_Fl( (diff(Curl(mu,sigma), R2[eta1]) + diff(Curl(mu, eta1), R2[sigma])), 1)

def G2s1s(mu, nu1, beta):
    return 0.5*b*FlFl( diff( G(mu, beta), R1[nu1]) + diff( G(nu1, beta), R1[mu]), 1,0)

def G2s2a(mu, nu1, sigma):
    return 0.25*b**2*Fl( (diff(tilde_Curl(mu,sigma), R1[nu1]) + diff(tilde_Curl(nu1, sigma), R1[mu])), 1)

def G2s2s(mu, nu1, sigma, eta1):
    return 0.25*b**2*FlFl( diff( G(mu,sigma), R1[nu1], R2[eta1]) + diff( G(nu1,sigma), R1[mu], R2[eta1])
                          +  diff( G(mu,eta1), R1[nu1], R2[sigma]) + diff( G(nu1,eta1), R1[mu], R2[sigma]), 1,1)

def G2s3t(mu, nu1, eta2):
    return -0.1*b**3*Fl( Laplacian( diff( G(mu, eta2), R1[nu1]) + diff( G(nu1, eta2), R1[mu]), R2), 1)

def G2s3a(mu, nu1, sigma, eta2):
    return 1/3*b**3*Fl( diff( diff( tilde_Curl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(mu, beta), R2), (beta,0,2)), R1[nu1]) 
                       + diff( diff( tilde_Curl(nu1, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(nu1, beta), R2), (beta,0,2)), R1[mu]), 1)

def G3a2s(mu, nu2, sigma, nu1):
    return 0.25*b**3*tilde_Fl( diff( diff( Curl(tau, beta), R2[kappa1]) + diff( Curl(tau, kappa1), R2[beta]), R1[gamma2])
                              + diff( diff( Curl(gamma2, beta), R2[kappa1]) + diff( Curl(gamma2, kappa1), R2[beta]), R1[tau]), 1)

def G3a3a(mu, nu2, sigma, eta2):
    return 1/3*b**4*( diff( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(mu, beta), R2), (beta,0,2)), R1[nu2]) 
                       + diff( diff( CurlCurl(nu2, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(nu2, beta), R2), (beta,0,2)), R1[mu]) )

def G1s3t(alpha, eta2):
    return -0.2*b**2*Fl( Laplacian( G(alpha, eta2), R2), 0)

def G1s3a(alpha, sigma, eta2):
    return 2/3*b**2*Fl( diff( tilde_Curl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2)), 0)

def G2a3a(mu, sigma, eta2):
    return 2/3*b**3*( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( Curl(mu,beta), R2), (beta,0,2)) )

def G3a1s(mu, nu2, beta):
    return 0.5*b**2*tilde_Fl( diff( Curl(mu,beta), R1[nu2]) + diff( Curl(nu2,beta), R1[mu]), 0)

def G3a2a(mu, nu2, sigma):
    return 0.25*b**3*( diff( CurlCurl(mu,sigma), R1[nu2]) + diff( CurlCurl(nu2,sigma), R1[mu]) )


def g13sSym(alpha, sigma, eta1, eta2):
    return 1/3*( diff( G(alpha, sigma), R2[eta1], R2[eta2]) + diff( G(alpha, eta1), R2[eta2], R2[sigma]) + diff( G(alpha, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( G(alpha, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( G(alpha, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( G(alpha, eta1), R2) )

def G1s3s(alpha, sigma, eta1, eta2):
    return b**2*FlFl( g13sSym(alpha, sigma, eta1, eta2), 0, 2)

def G2a3s(mu, sigma, eta1, eta2):
    return b**3*tilde_Fl( 1/3*( diff( Curl(mu, sigma), R2[eta1], R2[eta2]) + diff( Curl(mu, eta1), R2[eta2], R2[sigma]) + diff( Curl(mu, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( Curl(mu, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( Curl(mu, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( Curl(mu, eta1), R2) ), 2)


def g3s1Sym(alpha, gamma1, gamma2, beta): ##screwed up alpha --> mu etc indexing, so caution with 3s modes below! 
    return 1/3*( diff( G(alpha, beta), R1[gamma1], R1[gamma2]) + diff( G(gamma1, beta), R1[gamma2], R1[alpha]) + diff( G(gamma2, beta), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( G(gamma2, beta), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( G(alpha, beta), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( G(gamma1, beta), R1) )

def G3s1s(mu, nu1, nu2, beta):
    return b**2*FlFl( g3s1Sym(mu, nu1, nu2, beta), 2,0)

def G3s2a(alpha, gamma1, gamma2, sigma):
    return 0.5*b**3*Fl( 1/3*( diff( tilde_Curl(alpha, sigma), R1[gamma1], R1[gamma2]) + diff( tilde_Curl(gamma1, sigma), R1[gamma2], R1[alpha]) 
                             + diff( tilde_Curl(gamma2, sigma), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( tilde_Curl(gamma2, sigma), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( tilde_Curl(alpha, sigma), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( tilde_Curl(gamma1, sigma), R1) ), 2)

def g2s1Sym(mu, nu1, sigma):
    return 0.5*(diff( G(mu, sigma), R1[nu1]) + diff( G(nu1, sigma), R1[mu]))

def G2s3s(mu, nu1, sigma, eta1, eta2):
    return b**3*FlFl( 1/3*( diff( g2s1Sym(mu, nu1, sigma), R2[eta1], R2[eta2]) + diff( g2s1Sym(mu, nu1, eta1), R2[eta2], R2[sigma]) 
                           + diff( g2s1Sym(mu, nu1, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( g2s1Sym(mu, nu1, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( g2s1Sym(mu, nu1, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( g2s1Sym(mu, nu1, eta1), R2) ), 1,2)

def g3a1sSym(mu, nu2, sigma):
    return 0.5*( diff( Curl(mu,sigma), R1[nu2]) + diff( Curl(nu2,sigma), R1[mu]) )

def G3a3s(mu, nu2, sigma, eta1, eta2):
    return b**4*tilde_Fl( 1/3*( diff( g3a1Sym(mu, nu2, sigma), R2[eta1], R2[eta2]) + diff( g3a1Sym(mu, nu2, eta1), R2[eta2], R2[sigma]) 
                           + diff( g3a1Sym(mu, nu2, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( g3a1Sym(mu, nu2, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( g3a1Sym(mu, nu2, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( g3a1Sym(mu, nu2, eta1), R2) ), 2)


def g12sSym(alpha, sigma, eta1):
    return 0.5*( diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma]) )

def G3s2s(alpha, gamma1, gamma2, sigma, eta1):
    return b**3*FlFl( 1/3*( diff( g12sSym(alpha, sigma, eta1), R1[gamma1], R1[gamma2]) + diff( g12sSym(gamma1, sigma, eta1), R1[gamma2], R1[alpha]) 
                           + diff( g12sSym(gamma2, sigma, eta1), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g12sSym(gamma2, sigma, eta1), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g12sSym(alpha, sigma, eta1), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g12sSym(gamma1, sigma, eta1), R1) ), 2,1)


def g13aSym( alpha, sigma, eta2): 
    return diff( tilde_Curl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2))

def G3s3a(alpha, gamma1, gamma2, sigma, eta2):
    return 2/3*b**4*Fl( 1/3*( diff( g13aSym(alpha, sigma, eta2), R1[gamma1], R1[gamma2]) + diff( g13aSym(gamma1, sigma, eta2), R1[gamma2], R1[alpha]) 
                           + diff( g13aSym(gamma2, sigma, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13aSym(gamma2, sigma, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13aSym(alpha, sigma, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13aSym(gamma1, sigma, eta2), R1) ), 2)

def g13tSym(alpha, eta2):
    return Laplacian( G(alpha, eta2), R2)

def G3s3t(alpha, gamma1, gamma2, eta2):
    return -0.2*b**4*Fl( 1/3*( diff( g13tSym(alpha, eta2), R1[gamma1], R1[gamma2]) + diff( g13tSym(gamma1, eta2), R1[gamma2], R1[alpha]) 
                              + diff( g13tSym(gamma2, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13tSym(gamma2, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13tSym(alpha, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13tSym(gamma1, eta2), R1) ), 2)

def G3s3s(alpha, gamma1, gamma2, sigma, eta1, eta2):
    return b**4*FlFl( 1/3*( diff( g13sSym(alpha, sigma, eta1, eta2), R1[gamma1], R1[gamma2]) 
                           + diff( g13sSym(gamma1, sigma, eta1, eta2), R1[gamma2], R1[alpha]) 
                           + diff( g13sSym(gamma2, sigma, eta1, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13sSym(gamma2, sigma, eta1, eta2), R1) 
        + KroneckerDelta[gamma1, gamma2]*Laplacian( g13sSym(alpha, sigma, eta1, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13sSym(gamma1, sigma, eta1, eta2), R1) ), 2,2)

def G2a3t(mu, eta2):
    return -0.2*b**3*Laplacian( Curl(mu, eta2), R2)

def G3a3t(mu, nu2, eta2):
    return -0.2*b**4*Laplacian( g3a1sSym(mu, nu2, eta2), R2)


def k12sSym(alpha, sigma, eta1):
    return 0.5*( diff( G(alpha, eta1), R2[sigma]) + diff( G(alpha, sigma), R2[eta1]) )

def K1s2s(alpha, sigma, eta1):
    return 8*pi*eta*b**2/3*FlFl( k12sSym(alpha, sigma, eta1), 0,1)

def K1s3t(alpha, eta2):
    return -6*pi*eta*b**3/25*Fl( Laplacian( G(alpha, eta2), R2), 0)

def K2a2s(mu, sigma, eta1):
    return 4*pi*eta*b**3/3*tilde_Fl( diff( Curl(mu, eta1), R2[sigma]) + diff( Curl(mu, sigma), R2[eta1]), 1)


def k2s1Sym(mu, nu1, sigma):
    return 0.5*( diff( G(mu, sigma), R1[nu1]) + diff( G(nu1, sigma), R1[mu]))

def K2s2s(mu, nu1, sigma, eta1):
    return 4*pi*eta*b**3/3*FlFl( diff( k2s1Sym(mu, nu1, eta1), R2[sigma]) + diff( k2s1Sym(mu, nu1, sigma), R2[eta1]), 1,1)

def K2s3t(mu, nu1, eta2):
    return -6*pi*eta*b**4/25*Fl( 0.5*( diff( Laplacian(G(mu, eta2), R2), R1[nu1]) + diff( Laplacian(G(nu1, eta2), R2), R1[mu])), 1)

def K3a2s(mu, nu2, sigma, eta1):
    return 2*pi*eta*b**4/3*tilde_Fl( diff( diff( Curl(mu, eta1), R2[sigma]) + diff( Curl(mu, sigma), R2[eta1]), R1[nu2])
                                    + diff( diff( Curl(nu2, eta1), R2[sigma]) + diff( Curl(nu2, sigma), R2[eta1]), R1[mu]), 1)

def K3s2s(alpha, gamma1, gamma2, sigma, eta1): ## is this correct?
    return 8*pi*eta*b**4/3*FlFl( 1/3*( diff( k12sSym(alpha, sigma, eta1), R1[gamma1], R1[gamma2]) + diff( k12sSym(gamma1, sigma, eta1), R1[gamma2], R1[alpha]) 
                           + diff( k12sSym(gamma2, sigma, eta1), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( k12sSym(gamma2, sigma, eta1), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( k12sSym(alpha, sigma, eta1), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( k12sSym(gamma1, sigma, eta1), R1) ), 2,1) 

def K3s3t(alpha, gamma1, gamma2, eta2): ##up to factor same as G3s3t
    return -6*pi*eta*b**5/25*Fl( 1/3*( diff( g13tSym(alpha, eta2), R1[gamma1], R1[gamma2]) + diff( g13tSym(gamma1, eta2), R1[gamma2], R1[alpha]) 
                              + diff( g13tSym(gamma2, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13tSym(gamma2, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13tSym(alpha, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13tSym(gamma1, eta2), R1) ), 2)

def K2a3t(mu, eta2):
    return -6*pi*eta*b**4/25* Laplacian( Curl(mu, eta2), R2)

def K3a3t(mu, nu2, eta2):
    return -6*pi*eta*b**5/25*Laplacian( g3a1sSym(mu, nu2, eta2), R2)

In [8]:
## Really slow, can return to use 'summation' now that rM is an Array? Would that be quicker? instead of silly for loop
## Nope, for loop is at least as fast 

In [9]:
## Need Curl argument G? Def need it in Laplacian and Fl, so it is flexible. But Curl never taken of different function. Curl always only taken of G directly. 

In [10]:
## K^(l sigma, l sigma) is same as G with different pre factors, so use this symmetry

In [16]:
## try to parallelise G1s1s matrix computation - possible over many computers from notebook? 
## then try to implement FTS Stokesian dynamics first to test code? 

### Matrix assembly

In [17]:
%%time
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = G1s1s(i, j) ##do not simplify, costs lots of time

CPU times: user 2min 13s, sys: 7.96 ms, total: 2min 13s
Wall time: 2min 13s


In [ ]:
## for larger matrices, this might a) need parallel computing and b) dump memory once done. 
## Check whether this is actually working and just taking long or whether it runs into some random problem.
## G3s3s: after ~1h it was computing the fourth term in FlFl propto LaplacianLaplacian, so might take up to 2h per evaluation, was at 3% Memory usage 
## G2s2s: 4 min for 1 element

In [14]:
%%time
bla = G2s2s(0,0, 0,0)

CPU times: user 4min 11s, sys: 51.8 ms, total: 4min 11s
Wall time: 4min 11s


In [15]:
simplify(bla.subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h }))

b**2*(-0.02109375*b**4*theta + b**2*h**2*(0.05625*theta + 0.0140625) - h**4*(0.0546875*theta + 0.015625))/(pi*eta*h**7*(theta + 1))

In [13]:
simplify(bla.subs({x1-x2:xij, y1-y2:yij, z1+z2:dzij})) ##replacing composite variables does not work well

b*(-4*b**2*dzij**3*theta - 4*b**2*dzij*theta*xij**2 + 16*b**2*dzij*theta*yij**2 - 2*dzij**5*theta + 2*dzij**5 + 10*dzij**4*theta*z2 - 4*dzij**3*theta*xij**2 + 3*dzij**3*theta*y1*yij - 3*dzij**3*theta*y2*yij - 4*dzij**3*theta*yij**2 - 6*dzij**3*theta*z1*z2 - 12*dzij**3*theta*z2**2 + 4*dzij**3*xij**2 - 3*dzij**3*y1*yij + 3*dzij**3*y2*yij + 4*dzij**3*yij**2 + 14*dzij**2*theta*xij**2*z2 - 30*dzij**2*theta*y1**2*z2 + 60*dzij**2*theta*y1*y2*z2 - 12*dzij**2*theta*y1*yij*z2 - 30*dzij**2*theta*y2**2*z2 + 12*dzij**2*theta*y2*yij*z2 - 3*dzij**2*theta*yij**2*z1 + 11*dzij**2*theta*yij**2*z2 + 6*dzij**2*theta*z1*z2**2 + 6*dzij**2*theta*z2**3 + 3*dzij**2*yij**2*z1 + 3*dzij**2*yij**2*z2 - 2*dzij*theta*xij**4 + 3*dzij*theta*xij**2*y1*yij - 3*dzij*theta*xij**2*y2*yij - 4*dzij*theta*xij**2*yij**2 - 6*dzij*theta*xij**2*z1*z2 - 12*dzij*theta*xij**2*z2**2 + 3*dzij*theta*y1*yij**3 + 30*dzij*theta*y1*yij*z1*z2 + 60*dzij*theta*y1*yij*z2**2 - 3*dzij*theta*y2*yij**3 - 30*dzij*theta*y2*yij*z1*z2 - 60*dzij*theta*y